<a href="https://colab.research.google.com/github/nptan2005/spark401_colab/blob/main/notebooks/Spark_lab_instrodure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y openjdk-17-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jre session-migration x11-utils
Suggested packages:
  gvfs libxt-doc openjdk-17-demo openjdk-17-source visualvm mesa-utils
The following NEW packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jdk openjdk-17-jr

## 🎯 Mục tiêu luyện Spark trên Google Colab:
> *	Chạy Spark 4.0.1 thật (không giả lập)
> *	Viết PySpark giống production
> *	**Luyện:**
>>-	DataFrame API
>>-	Transformation vs Action
>>-	Partition / Shuffle
>>-	Basic optimization mindset
>>-	Không phụ thuộc GCP quota / Dataproc


---

  ## 🧱 Kiến trúc khi chạy Spark trên Colab



```mermaid
flowchart LR
    A[Google Colab Notebook] --> B[Local Spark 4.0.1]
    B --> C[In-memory DataFrame]
    B --> D[Local Disk /content]
```
> ⚠️ Spark chạy local mode, không phải cluster – nhưng API & tư duy y hệt Dataproc

---

### 🔹 STEP 1 – Tạo Notebook Colab
1.	Mở: https://colab.research.google.com
2.	New Notebook
3.	Runtime → Python 3
4.	(Optional) Runtime → Change runtime → High-RAM nếu có

---

### 🔹 STEP 2 – Cài Java (BẮT BUỘC)

Spark cần Java ≥ 11



```python
!apt-get install -y openjdk-17-jdk
```

Kiểm tra:

```python
!java -version
```

In [ ]:
!java -version

openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment (build 17.0.17+10-Ubuntu-122.04)
OpenJDK 64-Bit Server VM (build 17.0.17+10-Ubuntu-122.04, mixed mode, sharing)


### 🔹 STEP 3 – Cài Spark 4.0.1

```python
!wget https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
!tar xf spark-4.0.1-bin-hadoop3.tgz
```

In [ ]:
!wget https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
!tar xf spark-4.0.1-bin-hadoop3.tgz

--2025-12-29 06:06:53--  https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548955321 (524M) [application/x-gzip]
Saving to: ‘spark-4.0.1-bin-hadoop3.tgz’

spark-4.0.1-bin-had 100%[===================>] 523.52M  24.3MB/s    in 23s     

2025-12-29 06:07:17 (22.7 MB/s) - ‘spark-4.0.1-bin-hadoop3.tgz’ saved [548955321/548955321]



### 🔹 STEP 4 – Set ENV cho Spark

```python
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"
os.environ["PATH"] += ":/content/spark-4.0.1-bin-hadoop3/bin"
```

kiểm tra:

```python
!spark-submit --version
```
>👉 Phải thấy: Spark version 4.0.1

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"
os.environ["PATH"] += ":/content/spark-4.0.1-bin-hadoop3/bin"

In [ ]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 4.0.1
      /_/
                        
Using Scala version 2.13.16, OpenJDK 64-Bit Server VM, 17.0.17
Branch HEAD
Compiled by user runner on 2025-09-02T03:10:51Z
Revision 29434ea766b0fc3c3bf6eaadb43a8f931133649e
Url https://github.com/apache/spark
Type --help for more information.


### 🔹 STEP 5 – Cài PySpark tương thích

```bash
!pip install pyspark==4.0.1
```

In [ ]:
!pip install pyspark==4.0.1

### 🔹 STEP 6 – Tạo SparkSession (chuẩn production mindset)

```python
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Spark401-Colab-Training")
    .master("local[*]")
    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

spark
```

In [ ]:
# Tạo SparkSession
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Spark401-Colab-Training")
    .master("local[*]")
    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

spark

### 🔹 STEP 7 – Bài tập Spark CƠ BẢN (RẤT QUAN TRỌNG)

#### 7.1 Tạo DataFrame:

```python
data = [
    (1, "A", 100),
    (2, "A", 200),
    (3, "B", 300),
    (4, "B", 400),
    (5, "C", 500),
]

df = spark.createDataFrame(data, ["id", "group", "amount"])
df.show()
```

In [ ]:
# Tạo DataFrame
data = [
    (1, "A", 100),
    (2, "A", 200),
    (3, "B", 300),
    (4, "B", 400),
    (5, "C", 500),
]

df = spark.createDataFrame(data, ["id", "group", "amount"])
df.show()

+---+-----+------+
| id|group|amount|
+---+-----+------+
|  1|    A|   100|
|  2|    A|   200|
|  3|    B|   300|
|  4|    B|   400|
|  5|    C|   500|
+---+-----+------+



#### 7.2 Transformation vs Action:

```python
df2 = df.filter(df.amount > 200)   # transformation
```

Chưa chạy gì cả 👇

```python
df2.explain()
```

Chạy action:

```python
df2.show()
```

> 👉 Key mindset: Spark chỉ chạy khi có action

In [ ]:
# Transformation vs Action
df2 = df.filter(df.amount > 200)   # transformation

In [ ]:
df2.explain()

== Physical Plan ==
*(1) Filter (isnotnull(amount#2L) AND (amount#2L > 200))
+- *(1) Scan ExistingRDD[id#0L,group#1,amount#2L]




In [ ]:
# action
df2.show()

+---+-----+------+
| id|group|amount|
+---+-----+------+
|  3|    B|   300|
|  4|    B|   400|
|  5|    C|   500|
+---+-----+------+



#### 7.3 GroupBy + Aggregation

In [ ]:
# GroupBy + Aggregation
from pyspark.sql.functions import sum, avg

df.groupBy("group").agg(
    sum("amount").alias("total_amount"),
    avg("amount").alias("avg_amount")
).show()

+-----+------------+----------+
|group|total_amount|avg_amount|
+-----+------------+----------+
|    A|         300|     150.0|
|    B|         700|     350.0|
|    C|         500|     500.0|
+-----+------------+----------+



#### 7.4 Partition & Shuffle (cực quan trọng cho Dataproc sau này)

In [ ]:
# Partition & Shuffle (cực quan trọng cho Dataproc sau này)

df.rdd.getNumPartitions()

2

In [ ]:
df.repartition(4, "group").rdd.getNumPartitions()

4

#### 7.5 Write Data (giống Bronze/Silver)

In [ ]:
# Write Data (giống Bronze/Silver)
df.write.mode("overwrite").parquet("/content/output/orders")

In [ ]:
spark.read.parquet("/content/output/orders").show()

+---+-----+------+
| id|group|amount|
+---+-----+------+
|  3|    B|   300|
|  4|    B|   400|
|  5|    C|   500|
|  1|    A|   100|
|  2|    A|   200|
+---+-----+------+



## 🧠 1️⃣ Shuffle là gì

### 🔹 Định nghĩa ngắn gọn

**Shuffle** là quá trình Spark phải di chuyển dữ liệu giữa các executor/partition để sắp xếp lại dữ liệu.

> 👉 Shuffle = chậm + tốn CPU + tốn network + tốn tiền (trên cloud)

### 🔹 Khi nào shuffle xảy ra:

|Operation|Có shuffle không?|
|---------|-----------------|
|filter|❌ Không|
|select|❌ Không|
|withColumn|❌ Không|
|groupBy|✅ Có|
|join (thường)|✅ Có|
|orderBy|✅ Có|
|distinct|✅ Có|
|repartition()|✅ Có|
|coalesce()|❌ Không (nếu giảm partition)|

---

### 🔹 Ví dụ trực quan

```mermaid
flowchart LR
    A[Partition 1] -->|shuffle| D[New Partition A]
    B[Partition 2] -->|shuffle| E[New Partition B]
    C[Partition 3] -->|shuffle| F[New Partition C]
```

> 👉 Spark phải gom dữ liệu cùng key về cùng partition

### 🔹 Quy tắc vàng

> ❗ Ít shuffle = job chạy nhanh hơn rất nhiều
